# Japanese news classification with TFhub_BERT_201904 


### Evaluation results 
* eval_accuracy = 0.92424244
* eval_loss = 0.36567232
* global_step = 198
* loss = 0.53717786

finetuned with data of livedoor news corpus (training 2114 samples, test 529 samples)

## 1.  Set up Colab TPU running environment, connect TPU for Google Cloud storage

In [0]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.21.23.58:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 5295049173315565221),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 3756047874147407878),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 5497665024372258704),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 17840213422322543734),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 4940486551393530508),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 4727922890511607476),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 1349115528840877436),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 4252107389219669688),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 1106392262138910

## 2. Import multi-language BERT 

In [0]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

W0402 14:02:34.130271 140467120924544 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


## 3. Prepare for training

*  Specify task and put data (livedoor news courpus) into Goolge storage
*  Specify BERT pretrained model
*  Specify Goolge storage bucket, create output directory for model checkpoints and eval results



In [0]:
TASK = 'jp'
BUCKET = 'berthub04'

TASK_DATA_DIR = 'gs://{}/bert/dataset'.format(BUCKET)
!gsutil ls $TASK_DATA_DIR

BERT_MODEL = 'multi_cased_L-12_H-768_A-12' 
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

BERT_PRETRAINED_DIR = 'gs://{}/bert-tfhub/pre_trained/{}'.format(BUCKET, BERT_MODEL)
!gsutil ls $BERT_PRETRAINED_DIR
 
OUTPUT_DIR = 'gs://{}/bert-tfhub/models/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)

gs://berthub04/bert/dataset/
gs://berthub04/bert/dataset/dev.tsv
gs://berthub04/bert/dataset/train.tsv
CommandException: One or more URLs matched no objects.


## 4. Set new task for classification

In [0]:
class JpProcessor(run_classifier.ColaProcessor):
  def get_labels(self):
    return ["0", "1", "2"]
   
  def _create_examples(self, lines, set_type):
    examples = []
    for (i, line) in enumerate(lines):
      if set_type == "test" and i == 0:
        continue
      guid = "%s-%s" % (set_type, i)
      if set_type == "test":
        text_a = tokenization.convert_to_unicode(line[1])
        label = "0"
      else:
        text_a = tokenization.convert_to_unicode(line[1])
        label = tokenization.convert_to_unicode(line[0])
      examples.append(
          run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    return examples


## 5. Create functions for training

In [0]:
def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

PREDICT_BATCH_SIZE = 8


In [0]:
# Model Hyper Parameters
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
WARMUP_PROPORTION = 0.1
MAX_SEQ_LENGTH = 128

# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
ITERATIONS_PER_LOOP = 1000
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

processors = {
  "cola": run_classifier.ColaProcessor,
  "mnli": run_classifier.MnliProcessor,
  "mrpc": run_classifier.MrpcProcessor,
  "jp": JpProcessor, 
}

processor = processors[TASK.lower()]()
label_list = processor.get_labels()
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)

tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 1000

train_examples = processor.get_train_examples(TASK_DATA_DIR)
num_train_steps = int(
    len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)


In [0]:
# Force TF Hub writes to the GS bucket we provide.
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR

model_fn = run_classifier_with_tfhub.model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  bert_hub_module_handle=BERT_MODEL_HUB
)

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)


W0402 14:03:05.915925 140467120924544 estimator.py:1924] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7fc0c9de0a60>) includes params argument, but params are not passed to Estimator.


INFO:tensorflow:Using config: {'_model_dir': 'gs://berthub04/bert-tfhub/models/jp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.21.23.58:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc0e380d4e0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.21.23.58:8470', '_evaluation_master': 'grpc://10.21.23.58:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_

I0402 14:03:05.925165 140467120924544 estimator.py:201] Using config: {'_model_dir': 'gs://berthub04/bert-tfhub/models/jp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.21.23.58:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc0e380d4e0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.21.23.58:8470', '_evaluation_master': 'grpc://10.21.23.58:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per

INFO:tensorflow:_TPUContext: eval_on_tpu True


I0402 14:03:05.931272 140467120924544 tpu_context.py:202] _TPUContext: eval_on_tpu True


## 6. Start training

In [0]:
# Train the model
def model_train(estimator):
  print('MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...')
  # We'll set sequences to be at most 128 tokens long.
  train_features = run_classifier.convert_examples_to_features(
      train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_examples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))


In [0]:
tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)

Instructions for updating:
Colocations handled automatically by placer.


W0402 14:03:41.945940 140467120924544 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0402 14:03:45.200741 140467120924544 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [0]:
model_train(estimator_from_tfhub)

MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...
INFO:tensorflow:Writing example 0 of 2114


I0402 14:03:46.423940 140467120924544 run_classifier.py:774] Writing example 0 of 2114


INFO:tensorflow:*** Example ***


I0402 14:03:46.432276 140467120924544 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: train-0


...

INFO:tensorflow:Loss for final step: 0.0009960537.


I0402 14:07:06.049680 140467120924544 estimator.py:359] Loss for final step: 0.0009960537.


INFO:tensorflow:training_loop marked as finished


I0402 14:07:06.053042 140467120924544 error_handling.py:93] training_loop marked as finished


***** Finished training at 2019-04-02 14:07:06.060137 *****


## 7. Evaluate the result

In [0]:
def model_eval(estimator):
  # Eval the model.
  eval_examples = processor.get_dev_examples(TASK_DATA_DIR)
  eval_features = run_classifier.convert_examples_to_features(
      eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(eval_examples)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write("%s = %s\n" % (key, str(result[key])))


In [0]:
model_eval(estimator_from_tfhub)

INFO:tensorflow:Writing example 0 of 529


I0402 14:07:06.374649 140467120924544 run_classifier.py:774] Writing example 0 of 529


INFO:tensorflow:*** Example ***


I0402 14:07:06.382050 140467120924544 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: dev-0


...

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 198: gs://berthub04/bert-tfhub/models/jp/model.ckpt-198


I0402 14:07:53.872822 140467120924544 estimator.py:2039] Saving 'checkpoint_path' summary for global step 198: gs://berthub04/bert-tfhub/models/jp/model.ckpt-198


INFO:tensorflow:evaluation_loop marked as finished


I0402 14:07:54.362071 140467120924544 error_handling.py:93] evaluation_loop marked as finished


***** Finished evaluation at 2019-04-02 14:07:54.369090 *****
***** Eval results *****
  eval_accuracy = 0.92424244
  eval_loss = 0.36567232
  global_step = 198
  loss = 0.53717786


## 8. Make predictions based on the model

In [0]:
def model_predict(estimator):
  # Make predictions on a subset of eval examples
  prediction_examples = processor.get_dev_examples(TASK_DATA_DIR)[:PREDICT_BATCH_SIZE]
  input_features = run_classifier.convert_examples_to_features(prediction_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
  predictions = estimator.predict(predict_input_fn)

  for example, prediction in zip(prediction_examples, predictions):
    print('text_a: %s\ntext_b: %s\nlabel:%s\nprediction:%s\n' % (example.text_a, example.text_b, str(example.label), prediction['probabilities']))


In [0]:
model_predict(estimator_from_tfhub) 

INFO:tensorflow:Writing example 0 of 8


I0402 14:07:54.890913 140467120924544 run_classifier.py:774] Writing example 0 of 8


INFO:tensorflow:*** Example ***


I0402 14:07:54.894056 140467120924544 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: dev-0


...

I0402 14:08:18.511876 140467120924544 tpu_estimator.py:540] Dequeue next (1) batch(es) of data from outfeed.


text_a: ROOKIES俳優も不信感、進歩なき大晦日格闘技イベント
text_b: None
label:2
prediction:[3.7701294e-04 9.9901187e-01 6.1105227e-04]

text_a: “夢を追う”子供と“夢を諦めさせる”親の気持ち
text_b: None
label:1
prediction:[3.7492788e-04 9.9895358e-01 6.7146617e-04]

text_a: 清水退団のユングベリに非難轟々
text_b: None
label:2
prediction:[2.7925576e-04 5.6199665e-04 9.9915874e-01]

text_a: 裁定が覆った柔道、加藤浩次は「会場はそれで納得したという感じ」
text_b: None
label:2
prediction:[2.5634217e-04 5.8220222e-04 9.9916148e-01]

text_a: 作業用BGMを探すのに最適！　お気に入りのアーティストを連続再生「Musictonic」
text_b: None
label:0
prediction:[9.9935657e-01 3.2108990e-04 3.2232815e-04]

text_a: Windowsトラブル時にデータを救出！Ubuntuの便利な機能アレコレ 【デジ通】
text_b: None
label:0
prediction:[9.9935275e-01 2.8334139e-04 3.6388921e-04]

text_a: アメリカ大統領選の裏側を暴く作品がゴールデン・グローブ賞にノミネート
text_b: None
label:1
prediction:[3.5810156e-04 9.9900955e-01 6.3236093e-04]

text_a: “飛ばないボール”で野球人気が回復か？
text_b: None
label:2
prediction:[2.3941860e-04 6.2059838e-04 9.9913996e-01]

INFO:tensorflow:prediction_loop marked as finished


I0402 14:08:23.475856 140467120924544 error_handling.py:93] prediction_loop marked as finished


INFO:tensorflow:Error recorded from infeed: Step was cancelled by an explicit call to `Session::Close()`.


I0402 14:08:23.476411 140465820337920 error_handling.py:70] Error recorded from infeed: Step was cancelled by an explicit call to `Session::Close()`.
W0402 14:08:23.479538 140467120924544 tf_logging.py:161] Reraising captured error
Exception ignored in: <generator object TPUEstimator.predict at 0x7fc0afe65830>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/tpu/python/tpu/tpu_estimator.py", line 2500, in predict
    rendezvous.raise_errors()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/tpu/python/tpu/error_handling.py", line 132, in raise_errors
    six.reraise(typ, value, traceback)
  File "/usr/local/lib/python3.6/dist-packages/six.py", line 693, in reraise
    raise value
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/tpu/python/tpu/error_handling.py", line 101, in catch_errors
    yield
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/tpu/python/tpu/tpu_estimator.py", line 

This code is solely for educational purpose. The code cannot be used for investments or busineeses in practice. TOSHI STATS Co., Ltd. and I do not accept any responsibility or liability for loss or damage occasioned to any person or property through using materials, instructions, methods, algorithm or ideas contained herein, or acting or refraining from acting as a result of such use. TOSHI STATS Co., Ltd. and I expressly disclaim all implied warranties, including merchantability or fitness for any particular purpose. There will be no duty on TOSHI STATS Co., Ltd. and me to correct any errors or defects in the codes and the software.¶